# Universidade Federal do Rio Grande do Norte


## Programa de Pós-Graduação em Engenharia Elétrica e de Computação
## EEC1509 - Aprendizagem de Máquina


# Group

## João Lucas Correia Barbosa de Farias

## Júlio Freire Peixoto Gomes


# Project 2 - Traffic Sign Recognition


## About the Project
This project is divided in 6 files including this one, where each one represents one step in the process of deploying a machine learning algorithm. In this case, we chose a Neural Network algorithm as Classifier. The goal is to explore learning, generalization and batch-normalization techniques and compare results.

The dataset has over 50k images of traffic signs. Our goal is to predict which sign a specific image refers to.


### The details about the dataset are shown below.

The German Traffic Sign Benchmark is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011.

*   Single-image, multi-class classification problem
*   More than 40 classes
*   More than 50,000 images in total
*   Large, lifelike database

For more information, visit:

https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Also, for each class, that is a respective shape, color and sign id's. They are describred as follows:



1.   Shape ID
  *   0: red
  *   1: blue
  *   2: yellow
  *   3: white
2.   Color ID
  *   0: triangle
  *   1: circle
  *   2: diamond
  *   3: hexagon
  *   4: inverse-triangle
3.   Sign ID
  *   float: value according to Ukranian Traffic Rule

## The dataset was taken from Kaggle:
https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009

# 1.0 Install and Load Libraries


In [ ]:
%%capture
# install wandb
!pip install wandb

In [ ]:
import wandb
import logging
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.metrics import geometric_mean_score
from sklearn.neural_network import MLPClassifier

import h5py
import os
from PIL import Image

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.layers import Activation

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy

# 2.0 Login to Weights & Biases

In [ ]:
# login to wandb
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 3.0 Data Preparation

In [ ]:
# ratio used to split train and test data
val_size = 0.1

# seed used to reproduce purposes
seed = 13

# name of the input artifact
artifact_input_name_train = "traffic_sign_recognition/train.h5:latest"
artifact_input_name_labels = "traffic_sign_recognition/train_labels.csv:latest"

# type of the artifact
artifact_type = "Train"

In [ ]:
# initiate wandb project
run = wandb.init(project="traffic_sign_recognition", job_type="train")

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

logger.info("Downloading and reading artifact...")
artifact_train = run.use_artifact(artifact_input_name_train)
artifact_labels = run.use_artifact(artifact_input_name_labels)

artifact_train_path = artifact_train.file()
artifact_labels_path = artifact_labels.file()

24-07-2022 01:53:37 Downloading and reading artifact...


In [ ]:
image_data = []

with h5py.File(artifact_train_path, 'r') as hf:
  images = list(hf.keys())
  for img in images:
    data = hf[img]
    data_array = np.array(data)
    image_data.append(np.array(data_array))

train = np.array(image_data)

print(f"train.shape: {train.shape}")

train.shape: (11755,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
train_labels = np.loadtxt(artifact_labels_path, delimiter=',')
print(f"train_labels.shape: {train_labels.shape}")

train_labels.shape: (11755,)


### 3.1 Split Data into Train and Validation Sets

In [ ]:
# we will split the train set into training and validation sets
logger.info("Spliting data into training and validation sets...")
x_train, x_val, y_train, y_val = train_test_split(train,
                                                  train_labels,
                                                  test_size=val_size,
                                                  random_state=seed,
                                                  shuffle=True)

24-07-2022 01:53:42 Spliting data into training and validation sets...


In [ ]:
logger.info("x train: {}".format(x_train.shape))
logger.info("y train: {}".format(y_train.shape))
logger.info("x val: {}".format(x_val.shape))
logger.info("y val: {}".format(y_val.shape))

24-07-2022 01:53:42 x train: (10579,)
24-07-2022 01:53:42 y train: (10579,)
24-07-2022 01:53:42 x val: (1176,)
24-07-2022 01:53:42 y val: (1176,)


### 3.2 Base Model Training

Number of pixels to resize images

In [ ]:
IMAGE_HEIGHT = 30     # pixels
IMAGE_WIDTH = 30      # pixels

Resize image and normalize values (from 0-255 to 0-1.0) for both x_train and x_val

In [ ]:
x_train_copy = []
x_val_copy = []

for img in x_train:
  image = Image.fromarray(img)
  image = image.resize((IMAGE_HEIGHT,IMAGE_WIDTH))
  img = np.array(image)
  x_train_copy.append(img)

for img in x_val:
  image = Image.fromarray(img)
  image = image.resize((IMAGE_HEIGHT,IMAGE_WIDTH))
  img = np.array(image)
  x_val_copy.append(img)

x_train_copy = np.array(x_train_copy)
x_val_copy = np.array(x_val_copy)

x_train_copy = x_train_copy/255  
x_val_copy = x_val_copy/255

In [ ]:
print(f"x_train_copy.shape: {x_train_copy.shape}")
print(f"x_val_copy.shape: {x_val_copy.shape}")

x_train_copy.shape: (10579, 30, 30, 3)
x_val_copy.shape: (1176, 30, 30, 3)


The LeNet-5 architecture was chosen as the model for our training.

In [ ]:
lenet5 = Sequential()

lenet5.add(Conv2D(6, (5,5), strides=1,  activation='tanh', input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3), padding='same')) #C1
lenet5.add(AveragePooling2D()) #S2
lenet5.add(Conv2D(16, (5,5), strides=1, activation='tanh', padding='valid')) #C3
lenet5.add(AveragePooling2D()) #S4
lenet5.add(Flatten()) #Flatten
lenet5.add(Dense(120, activation='tanh')) #C5
lenet5.add(Dense(84, activation='tanh')) #F6
lenet5.add(Dense(43, activation='softmax')) #Output layer

In [ ]:
lenet5.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy', 
               metrics='accuracy')

# training 
history = lenet5.fit(x=x_train_copy,
                    y=y_train,
                    batch_size=32,
                    epochs=6,
                    validation_data=(x_val_copy,y_val))

Epoch 1/6
331/331 [==============================] - 12s 35ms/step - loss: 2.0442 - accuracy: 0.4627 - val_loss: 0.9487 - val_accuracy: 0.7491
Epoch 2/6
331/331 [==============================] - 11s 34ms/step - loss: 0.6810 - accuracy: 0.8266 - val_loss: 0.4713 - val_accuracy: 0.8673
Epoch 3/6
331/331 [==============================] - 11s 34ms/step - loss: 0.3788 - accuracy: 0.9055 - val_loss: 0.3023 - val_accuracy: 0.9286
Epoch 4/6
331/331 [==============================] - 11s 34ms/step - loss: 0.2540 - accuracy: 0.9400 - val_loss: 0.2268 - val_accuracy: 0.9413
Epoch 5/6
331/331 [==============================] - 11s 34ms/step - loss: 0.1832 - accuracy: 0.9561 - val_loss: 0.1981 - val_accuracy: 0.9490
Epoch 6/6
331/331 [==============================] - 11s 34ms/step - loss: 0.1359 - accuracy: 0.9710 - val_loss: 0.1625 - val_accuracy: 0.9626


# 4.0 Pipeline

Class created to assist in normalizing the dataset array (from 0-255 to 0-1)

In [ ]:
class Normalize():
  def __init__(self, value=255):
    self.value = value

  def fit(self, X):
    return self

  def transform(self, X):
    return X/self.value

## 4.1 Feature Selector

The input to the pipeline is the x_train set. We will create a class whose sole goal is to return the object itself to be the beginning of the pipeline.

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X

In [ ]:
fs = FeatureSelector()
x_train_fs = fs.fit_transform(x_train)
x_train_fs[0]

array([[[143, 127, 142],
        [159, 160, 177],
        [146, 165, 183],
        ...,
        [ 40,  36,  45],
        [ 51,  45,  57],
        [ 60,  55,  71]],

       [[155, 143, 146],
        [148, 146, 148],
        [140, 148, 156],
        ...,
        [ 32,  32,  38],
        [ 32,  31,  35],
        [ 30,  30,  33]],

       [[139, 130, 127],
        [137, 132, 128],
        [136, 134, 133],
        ...,
        [ 39,  42,  48],
        [ 38,  41,  46],
        [ 37,  41,  45]],

       ...,

       [[ 31,  44,  47],
        [ 29,  41,  44],
        [ 30,  41,  44],
        ...,
        [ 28,  28,  30],
        [ 30,  28,  30],
        [ 31,  29,  30]],

       [[ 65,  71,  72],
        [ 67,  70,  71],
        [ 69,  69,  70],
        ...,
        [ 29,  30,  32],
        [ 30,  30,  31],
        [ 34,  33,  34]],

       [[ 47,  46,  49],
        [ 50,  44,  47],
        [ 51,  40,  43],
        ...,
        [ 29,  33,  33],
        [ 37,  38,  40],
        [ 44,  44,  47]]

## 4.2 Processing Numerical Features

In [ ]:
class NumericalTransformer(BaseEstimator, TransformerMixin):
    # normalize = False: no scaler
    # normalize = True: normalize RBG values
    def __init__(self, normalize=True, image_height=30, image_width=30):
        self.normalize = normalize
        self.image_height = image_height
        self.image_width = image_width
        self.scaler = None

    def fit(self, X, y=None):
        if self.normalize:
          self.scaler = Normalize(255)
          self.scaler.fit(X)
        return self

    # transforming numerical features
    def transform(self, X, y=None):
        X_copy = []

        for img in X:
          image = Image.fromarray(img)
          image = image.resize((self.image_height,self.image_width))
          img = np.array(image)
          X_copy.append(img)

        X_copy = np.array(X_copy)

        if self.normalize:
          X_copy = self.scaler.transform(X_copy)

        return X_copy

In [ ]:
# testing functions
fs = FeatureSelector()
x_train_fs = fs.fit_transform(x_train)
x_train_fs[0]

In [ ]:
# testing functions
nt = NumericalTransformer(normalize=True, image_height=30, image_width=30)
x_train_num = nt.fit_transform(x_train_fs)

print(f"x_train_num.shape: {x_train_num.shape}")
print(f"x_train_num[0]: {x_train_num[0]}")

## 4.3 Pipeline Creation

The pipeline will only contain the transformations necessary to treat the raw data and get it ready for the training. The neural network training is perfomed as a separate module. Both the pipeline and the NN model are exported to W&B. This way, in order to use our pipeline/model, one needs to pass the raw data through first and then feed it to the trained model to obtain the desired results.

In [ ]:
# model = 0 (min-max), 1 (z-score), 2 (no scaling)
normalize = True
IMAGE_HEIGHT = 30
IMAGE_WIDTH = 30

# defining the steps in the numerical pipeline
numerical_pipeline = Pipeline(steps=[('num_selector', FeatureSelector()),
                                     ('num_transformer', NumericalTransformer(normalize=normalize, image_height=IMAGE_HEIGHT, image_width=IMAGE_WIDTH))])

# combining numerical and categorical pieplines into one full big pipeline horizontally
full_pipeline_preprocessing = FeatureUnion(transformer_list=[
                                                             ('num_pipeline', numerical_pipeline)]
                                           )

In [ ]:
# testing pipeline
new_data = full_pipeline_preprocessing.fit_transform(x_train)
print(f"new_data.shape: {new_data.shape}")
print(f"new_data[0]: {new_data[0]}")

new_data.shape: (10579, 30, 30, 3)
new_data[0]: [[[0.56470588 0.50588235 0.56470588]
  [0.61960784 0.63137255 0.69411765]
  [0.57254902 0.65098039 0.71764706]
  ...
  [0.15294118 0.1372549  0.17254902]
  [0.19607843 0.17254902 0.21568627]
  [0.23137255 0.21176471 0.27058824]]

 [[0.60392157 0.56078431 0.56862745]
  [0.57254902 0.56862745 0.58039216]
  [0.54901961 0.58039216 0.61568627]
  ...
  [0.1254902  0.1254902  0.14901961]
  [0.1254902  0.12156863 0.1372549 ]
  [0.11764706 0.11764706 0.12941176]]

 [[0.54117647 0.50980392 0.49411765]
  [0.53333333 0.51372549 0.49803922]
  [0.52941176 0.52156863 0.51372549]
  ...
  [0.15294118 0.16470588 0.18823529]
  [0.14901961 0.16078431 0.18039216]
  [0.14509804 0.16078431 0.17647059]]

 ...

 [[0.11372549 0.16470588 0.17647059]
  [0.10588235 0.15294118 0.16470588]
  [0.10980392 0.15294118 0.16470588]
  ...
  [0.10980392 0.10980392 0.12156863]
  [0.11764706 0.10980392 0.11764706]
  [0.12156863 0.11372549 0.11764706]]

 [[0.25098039 0.2745098  0

In [ ]:
x_train = full_pipeline_preprocessing.fit_transform(x_train)
x_val = full_pipeline_preprocessing.transform(x_val)

print(f"x_train.shape: {x_train.shape}")
print(f"x_val.shape: {x_val.shape}")

x_train.shape: (10579, 30, 30, 3)
x_val.shape: (1176, 30, 30, 3)


In [ ]:
run.finish()

# 5.0 Training

## 5.1 Hyperparameter Tuning

In [ ]:
from wandb.keras import WandbCallback

In [ ]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    defaults = dict(
                layer_1_kernel_initializer = 'glorot_uniform',
                activation_function = 'tanh',
                optimizer = 'Adam',
                clipnorm = False,
                learning_rate = 0.001,
                loss = 'categorical_crossentropy',
                batch_size = 32,
                epochs = 5,
                dropout = False,
                early_stopping = False,
                batch_normalization = 0
                )

    
    # Initialize a new wandb run
    wandb.init(project="ppgeec-ml-jj/traffic_sign_recognition", config= defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config


    # neural network layers    
    lenet5 = Sequential()

    # testing the effects of batch-normalization (before activation function)
    if config.batch_normalization != 1:
      lenet5.add(Conv2D(6, (5,5), strides=1, kernel_initializer=config.layer_1_kernel_initializer, activation=config.activation_function, input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3), padding='same')) 
    else:
      lenet5.add(Conv2D(6, (5,5), strides=1, kernel_initializer=config.layer_1_kernel_initializer, input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3), padding='same')) 
      lenet5.add(BatchNormalization())
      lenet5.add(Activation(config.activation_function))

    # testing the effects of batch-normalization (after activation function)
    if config.batch_normalization == 2:
      lenet5.add(BatchNormalization())

    lenet5.add(AveragePooling2D()) 
    lenet5.add(Conv2D(16, (5,5), strides=1, activation=config.activation_function, padding='valid'))

    # testing the addition of dropout layers
    if config.dropout:
      lenet5.add(Dropout(rate=0.5))

    lenet5.add(AveragePooling2D()) #S4
    lenet5.add(Flatten()) #Flatten
    lenet5.add(Dense(120, activation=config.activation_function)) #C5

    # testing the addition of dropout layers
    if config.dropout:
      lenet5.add(Dropout(rate=0.5))

    lenet5.add(Dense(84, activation=config.activation_function)) #F6

    # testing the addition of dropout layers
    if config.dropout:
      lenet5.add(Dropout(rate=0.25))

    lenet5.add(Dense(43, activation='softmax')) #Output layer
    # end of neural network layers   

    # testing different loss functions
    loss = config.loss

    # Instantiate an accuracy metric.
    accuracy = Accuracy()


    # testing optimizer variance effect
    # testing learning rate variance 
    # testing the addition of gradient clipping to avoid exploding gradient
    if config.optimizer == 'Adam':
      if config.clipnorm:
        optimizer = Adam(learning_rate=config.learning_rate, clipnorm=1.0)
      else:
        optimizer = Adam(learning_rate=config.learning_rate)

    if config.optimizer == 'SGD':
      if config.clipnorm:
        optimizer = SGD(learning_rate=config.learning_rate, clipnorm=1.0)
      else:
        optimizer = SGD(learning_rate=config.learning_rate)

    if config.optimizer == 'RMSprop':
      if config.clipnorm:
        optimizer = RMSprop(learning_rate=config.learning_rate, clipnorm=1.0)
      else:
        optimizer = RMSprop(learning_rate=config.learning_rate)



    # configure the optimizer, loss, and metrics to monitor.
    lenet5.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) 

    # testing the effects of early stopping
    if config.early_stopping:
      es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) 
      lenet5.fit(x_train, y_train, 
              # testing different batch sizes
              batch_size=config.batch_size,
              # testing epochs variance effect
              epochs=config.epochs,
              validation_data=(x_val, y_val),
              # testing early stopping effect
              callbacks=[es, WandbCallback()]
              )
    else:
      lenet5.fit(x_train, y_train, 
            # testing different batch sizes
            batch_size=config.batch_size,
            # testing epochs variance effect
            epochs=config.epochs,
            validation_data=(x_val, y_val),
            callbacks=[WandbCallback()]
            ) 
    
    # model.save('modelo.h5')

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        # fix exploding gradient with relu (needs 'he_uniform')
        'layer_1_kernel_initializer': {
            'values': ['glorot_uniform', 'he_uniform']
        },
        # fix exploding gradient with relu
        'activation_function': {
            'values': ['tanh', 'relu']
        },
        # testing learning rate variance effect
        'learning_rate': {
            'min': -4,
            'max': -2,
            'distribution': 'log_uniform'
        },
        # testing optimizer variance effect
        'optimizer': {
            'values': ['Adam', 'RMSprop', 'SGD']
        },
        # testing the addition of gradient clipping to avoid exploding gradient
        'clipnorm': {
            'values': [True, False]
        },
        # testing epochs variance effect
        'epochs': {
            'values': [5,10,20]
        },
        # testing different batch sizes
        'batch_size': {
            'values': [16,32,64]
        },
        # testing different loss functions
        'loss': {
            'values': ['kullback_leibler_divergence',
                      'sparse_categorical_crossentropy']
        },
        # testing the addition of dropout layers
        'dropout': {
            'values': [True, False]
        },
        # testing the effects of early stopping
        'early_stopping': {
            'values': [True, False]
        },
        # testing the effects of batch-normalization
        # 0: no batch-normalization
        # 1: batch-normalization before activation function
        # 2: batch-normalization after activation function
        'batch_normalization': {
            'values': [0, 1, 2]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="traffic_sign_recognition")

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. learning_rate uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: ftpjniuf
Sweep URL: https://wandb.ai/ppgeec-ml-jj/traffic_sign_recognition/sweeps/ftpjniuf


There is a total of 15,552 combinations of the sweep configuration we created. To save time, we only try 100 of those and select the best one in terms of accuracy.

In [ ]:
wandb.agent(sweep_id, train, count=100)

## 5.2 Training Best Model

<font color="red">Important</font> to restart the colab to unlink a new experiment (run) with the last ```sweep``` experiment. 

```
Runtime >> Factory reset runtime
```
> Re-run all cells except for the ones corresponding to neural network training.








In [ ]:
# setup wandb
wandb.init(project="traffic_sign_recognition",
           config={
               "layer_1_kernel_initializer": 'glorot_uniform',
               "activation_function": 'tanh',
               "optimizer": 'SGD',
               "clipnorm": False,
               "learning_rate": 0.06645,
               "loss": 'sparse_categorical_crossentropy',
               "batch_size": 16,
               "epochs": 10,
               "dropout": False,
               "early_stopping": False,
               "batch_normalization": 2
           })
config = wandb.config

wandb: Currently logged in as: juliofreire (ppgeec-ml-jj). Use `wandb login --relogin` to force relogin


In [ ]:
# neural network layers    
lenet5 = Sequential()

# testing the effects of batch-normalization (before activation function)
if config.batch_normalization != 1:
  lenet5.add(Conv2D(6, (5,5), strides=1, kernel_initializer=config.layer_1_kernel_initializer, activation=config.activation_function, input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3), padding='same')) 
else:
  lenet5.add(Conv2D(6, (5,5), strides=1, kernel_initializer=config.layer_1_kernel_initializer, input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3), padding='same')) 
  lenet5.add(BatchNormalization())
  lenet5.add(Activation(config.activation_function))

# testing the effects of batch-normalization (after activation function)
if config.batch_normalization == 2:
  lenet5.add(BatchNormalization())

lenet5.add(AveragePooling2D()) 
lenet5.add(Conv2D(16, (5,5), strides=1, activation=config.activation_function, padding='valid'))

# testing the addition of dropout layers
if config.dropout:
  lenet5.add(Dropout(rate=0.5))

lenet5.add(AveragePooling2D()) #S4
lenet5.add(Flatten()) #Flatten
lenet5.add(Dense(120, activation=config.activation_function)) #C5

# testing the addition of dropout layers
if config.dropout:
  lenet5.add(Dropout(rate=0.5))

lenet5.add(Dense(84, activation=config.activation_function)) #F6

# testing the addition of dropout layers
if config.dropout:
  lenet5.add(Dropout(rate=0.25))

lenet5.add(Dense(43, activation='softmax')) #Output layer
# end of neural network layers   

# testing different loss functions
loss = config.loss

# Instantiate an accuracy metric.
accuracy = Accuracy()


# testing optimizer variance effect
# testing learning rate variance 
# testing the addition of gradient clipping to avoid exploding gradient
if config.optimizer == 'Adam':
  if config.clipnorm:
    optimizer = Adam(learning_rate=config.learning_rate, clipnorm=1.0)
  else:
    optimizer = Adam(learning_rate=config.learning_rate)

if config.optimizer == 'SGD':
  if config.clipnorm:
    optimizer = SGD(learning_rate=config.learning_rate, clipnorm=1.0)
  else:
    optimizer = SGD(learning_rate=config.learning_rate)

if config.optimizer == 'RMSprop':
  if config.clipnorm:
    optimizer = RMSprop(learning_rate=config.learning_rate, clipnorm=1.0)
  else:
    optimizer = RMSprop(learning_rate=config.learning_rate)



# configure the optimizer, loss, and metrics to monitor.
lenet5.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) 

# testing the effects of early stopping
if config.early_stopping:
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) 
  lenet5.fit(x_train, y_train, 
          # testing different batch sizes
          batch_size=config.batch_size,
          # testing epochs variance effect
          epochs=config.epochs,
          validation_data=(x_val, y_val),
          # testing early stopping effect
          callbacks=[es, WandbCallback()]
          )
else:
  lenet5.fit(x_train, y_train, 
        # testing different batch sizes
        batch_size=config.batch_size,
        # testing epochs variance effect
        epochs=config.epochs,
        validation_data=(x_val, y_val),
        callbacks=[WandbCallback()]
        ) 

lenet5.save('best_model.h5')

wandb.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


23-07-2022 19:53:13 From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/nn_ops.py:5214: tensor_shape_from_node_def_name (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/10
662/662 [==============================] - 16s 23ms/step - loss: 1.2515 - accuracy: 0.6729 - val_loss: 0.5148 - val_accuracy: 0.8690 - _timestamp: 1658606008.0000 - _runtime: 53.0000
Epoch 2/10
662/662 [==============================] - 15s 22ms/step - loss: 0.3646 - accuracy: 0.9143 - val_loss: 0.3203 - val_accuracy: 0.9260 - _timestamp: 1658606023.0000 - _runtime: 68.0000
Epoch 3/10
662/662 [==============================] - 15s 22ms/step - loss: 0.2096 - accuracy: 0.9545 - val_loss: 0.2112 - val_accuracy: 0.9498 - _timestamp: 1658606038.0000 - _runtime: 83.0000
Epoch 4/10
662/662 [==============================] - 15s 22ms/step - loss: 0.1399 - accuracy: 0.9707 - val_loss: 0.1845 - val_accuracy: 0.9515 - _timestamp: 1658606053.0000 - _runtime: 98.0000
Epoch 5/10
662/662 [==============================] - 15s 22ms/step - loss: 0.0945 - accuracy: 0.9831 - val_loss: 0.1712 - val_accuracy: 0.9541 - _timestamp: 1658606068.0000 - _runtime: 113.0000
Epoch 6/10
662/662 [=========

accuracy,▁▆▇▇██████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇█▇███
val_loss,█▅▃▃▂▂▁▁▁▁
GFLOPS,0.00076
accuracy,0.99934
best_epoch,9
best_val_loss,0.09141
epoch,9
loss,0.0208


INFO:tensorflow:Assets written to: best-model.model/assets


23-07-2022 20:01:47 Assets written to: best-model.model/assets


## 5.3 Export Best Model to WandB

In [ ]:
# types and names of the artifacts
artifact_type = "inference_artifact"
artifact_pipe = "pipeline"
artifact_model = "model.h5"

In [ ]:
logger.info("Dumping the artifacts to disk")
# Save the pipeline using joblib
joblib.dump(full_pipeline_preprocessing, artifact_pipe)

# Save the model using joblib
joblib.dump(best_model, artifact_model)

23-07-2022 13:11:17 Dumping the artifacts to disk


INFO:tensorflow:Assets written to: ram://71a042e8-e393-4d66-8894-802e0c73b02e/assets


23-07-2022 13:11:18 Assets written to: ram://71a042e8-e393-4d66-8894-802e0c73b02e/assets


['model']

In [ ]:
print(os.path.getsize('best_model.h5'))
print(os.path.getsize('model.h5'))
print(os.path.getsize('pipeline'))

295088
450845
505


In [ ]:
# Pipeline artifact
artifact = wandb.Artifact(artifact_pipe,
                          type=artifact_type,
                          description="A full pipeline composed of a Preprocessing Stage"
                          )

logger.info("Logging model artifact")
artifact.add_file(artifact_pipe)
run.log_artifact(artifact)

23-07-2022 13:13:03 Logging model artifact


In [ ]:
# Model artifact
artifact = wandb.Artifact(artifact_model,
                          type=artifact_type,
                          description="The trained model"
                          )

logger.info("Logging target enconder artifact")
artifact.add_file(artifact_model)
run.log_artifact(artifact)

23-07-2022 13:13:03 Logging target enconder artifact


In [ ]:
run.finish()